#Building an image retrieval system with deep features


#Fire up GraphLab Create

In [1]:
import graphlab

#Load the CIFAR-10 dataset

We will use a popular benchmark dataset in computer vision called CIFAR-10.  

(We've reduced the data to just 4 categories = {'cat','bird','automobile','dog'}.)

This dataset is already split into a training set and test set. In this simple retrieval example, there is no notion of "testing", so we will only use the training data.

In [27]:
image_train = graphlab.SFrame('image_train_data/')
image_test = graphlab.SFrame('image_test_data/')

#Computing deep features for our images

The two lines below allow us to compute deep features.  This computation takes a little while, so we have already computed them and saved the results as a column in the data you loaded. 

(Note that if you would like to compute such deep features and have a GPU on your machine, you should use the GPU enabled GraphLab Create, which will be significantly faster for this task.)

In [ ]:
#deep_learning_model = graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
#image_train['deep_features'] = deep_learning_model.extract_features(image_train)

In [3]:
image_train.head()

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


#Train a nearest-neighbors model for retrieving images using deep features

We will now build a simple image retrieval system that finds the nearest neighbors for any image.

In [4]:
knn_model = graphlab.nearest_neighbors.create(image_train,features=['deep_features'],
                                             label='id')

Starting brute force nearest neighbors model training.

#Use image retrieval model with deep features to find similar images

Let's find similar images to this cat picture.

In [5]:
cat = image_train[18:19]
cat['image'].show()

Canvas is accessible via web browser at the URL: http://localhost:39042/index.html
Opening Canvas in default web browser.


In [6]:
knn_model.query(cat)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 57.918ms     |

| Done         |         | 100         | 416.052ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,384,0.0,1
0,6910,36.9403137951,2
0,39777,38.4634888975,3
0,36870,39.7559623119,4
0,41734,39.7866014148,5


We are going to create a simple function to view the nearest neighbors to save typing:

In [7]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'],'id')

In [8]:
cat_neighbors = get_images_from_ids(knn_model.query(cat))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 37.454ms     |

| Done         |         | 100         | 385.787ms    |

+--------------+---------+-------------+--------------+

In [9]:
cat_neighbors['image'].show()

Canvas is updated and available in a tab in the default browser.


Very cool results showing similar cats.

##Finding similar images to a car

In [10]:
car = image_train[8:9]
car['image'].show()

Canvas is updated and available in a tab in the default browser.


In [11]:
get_images_from_ids(knn_model.query(car))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 50.149ms     |

| Done         |         | 100         | 414.842ms    |

+--------------+---------+-------------+--------------+

Canvas is updated and available in a tab in the default browser.


#Just for fun, let's create a lambda to find and show nearest neighbor images

In [12]:
show_neighbors = lambda i: get_images_from_ids(knn_model.query(image_train[i:i+1]))['image'].show()

In [13]:
show_neighbors(8)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 54.151ms     |

| Done         |         | 100         | 402.939ms    |

+--------------+---------+-------------+--------------+

Canvas is updated and available in a tab in the default browser.


In [15]:
show_neighbors(26)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 53.022ms     |

| Done         |         | 100         | 389.984ms    |

+--------------+---------+-------------+--------------+

Canvas is updated and available in a tab in the default browser.


In [16]:
show_neighbors(200)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 48.521ms     |

| Done         |         | 100         | 406.701ms    |

+--------------+---------+-------------+--------------+

Canvas is updated and available in a tab in the default browser.


In [22]:
dog_train = image_train.filter_by("dog", "label")

In [23]:
cat_train = image_train.filter_by("cat", "label")

In [24]:
bird_train = image_train.filter_by("bird", "label")

In [25]:
car_train = image_train.filter_by("automobile", "label")

In [28]:
dog_test = image_test.filter_by("dog", "label")

In [29]:
cat_test = image_test.filter_by("cat", "label")

In [30]:
bird_test = image_test.filter_by("bird", "label")

In [31]:
car_test = image_test.filter_by("automobile", "label")

In [32]:
dog_model = graphlab.nearest_neighbors.create(dog_train,features=['deep_features'],
                                             label='id')

Starting brute force nearest neighbors model training.

In [33]:
cat_model = graphlab.nearest_neighbors.create(cat_train,features=['deep_features'],
                                             label='id')

Starting brute force nearest neighbors model training.

In [34]:
bird_model = graphlab.nearest_neighbors.create(bird_train,features=['deep_features'],
                                             label='id')

Starting brute force nearest neighbors model training.

In [35]:
car_model = graphlab.nearest_neighbors.create(car_train,features=['deep_features'],
                                             label='id')

Starting brute force nearest neighbors model training.

In [36]:
cat_first = image_test[0:1]

In [37]:
cat_model.query(cat_first)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 41.312ms     |

| Done         |         | 100         | 169.928ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,16289,34.623719208,1
0,45646,36.0068799284,2
0,32139,36.5200813436,3
0,25713,36.7548502521,4
0,331,36.8731228168,5


In [38]:
dog_model.query(cat_first)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 43.944ms     |

| Done         |         | 100         | 198.59ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,16976,37.4642628784,1
0,13387,37.5666832169,2
0,35867,37.6047267079,3
0,44603,37.7065585153,4
0,6094,38.5113254907,5


In [47]:
closest_cat = cat_train[0:1]

In [48]:
closest_cat.show()

Canvas is updated and available in a tab in the default browser.


In [49]:
show_neighbors(1)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 35.867ms     |

| Done         |         | 100         | 420.042ms    |

+--------------+---------+-------------+--------------+

Canvas is accessible via web browser at the URL: http://localhost:39042/index.html
Opening Canvas in default web browser.
